In [71]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [72]:
# Read the Semeval data and return a dataframe
def read_semeval_data(filename):
    '''
    Description: reads Semantic Evaluation XML dataset and converts into a 
                 dataframffe
    Arguments:
                 filename: string with file path (including filename)
    Returns :    pandas data frame
    Important:   this function only puts positive and neutral reviews in dataframe                          
    '''
    f = open(filename, 'r')
    raw_training_data = f.read()
    xmldoc = BeautifulSoup(raw_training_data,'lxml-xml')
    sentences = xmldoc.Reviews.find_all('sentences')
    opinions = xmldoc.Reviews.find_all('Opinions')
    reviews = []
    for i in range(0,len(sentences)):
        record = {}
        entity_aspect_pairs = opinions[i].find_all('Opinion')
        for ea_pair in entity_aspect_pairs:
            ea = ea_pair.attrs['category']
            polarity = ea_pair.attrs['polarity']
            if(polarity == 'positive'):
                record[ea] = 1
            elif(polarity == 'negative'):
                record[ea] = -1
            else:
                record[ea] = 0
        
        #test=sentences[i].get_text()
        #r = re.compile(r'[^nmz]+')
        #r.sub('', test)
        #test2=re.sub(r'\W+â+€“ ', ' ', test)
        #print(test2)
        #record['TEXT'] = test2
        record['TEXT'] = sentences[i].get_text()
        reviews.append(record)
    #Create a dataframe
    df=pd.DataFrame(reviews)
    #Change order of the columns so that text appears first
    cols = df.columns.tolist()
    cols.sort()
    cols.reverse()
    df = df[cols]
    df.fillna(0, inplace=True)
    return df

In [73]:
df_training = read_semeval_data('data/train.xml')
df_testing = read_semeval_data('data/test.xml')

In [74]:
#Function to clean the text data
#Remove punctuations, newline characters and convert to lowercase.
#Note that we are not removing dot to mark sentence boundary
def clean_text_data(data):
    '''
    Description: Given text returns cleaned version
    Arguments:
                  data: string with raw review text
    Returns  :
                  cleaned: string with unwanted characters removed
    '''
    prog = re.compile('[\t\n\r\f\v\d\']', re.UNICODE)
    data = re.sub(prog, ' ', str(data)).lower()
    prog = re.compile('[!\"#$%&\'()*+\,-/:;<=>?@[\]^_`{|}~]', re.UNICODE)
    cleaned = re.sub(prog, ' ', data)
    return cleaned

In [75]:
#Function to decode and print output labels and polarity for a review
def output_to_labels(output):
    '''
     Description: Converts predicted output for a review into labels and polarity
     Arguments: output a numpy array
                
    '''
    labels = ['SERVICE#GENERAL', 'RESTAURANT#PRICES',
       'RESTAURANT#MISCELLANEOUS', 'RESTAURANT#GENERAL', 'LOCATION#GENERAL',
       'FOOD#STYLE_OPTIONS', 'FOOD#QUALITY', 'FOOD#PRICES',
       'DRINKS#STYLE_OPTIONS', 'DRINKS#QUALITY', 'DRINKS#PRICES',
       'AMBIENCE#GENERAL']
    for index in range(len(labels)):
        value = output[0,index]
        if(value == 1):
            print(labels[index],':',' ','positive')
        elif(value == -1):
            print(labels[index],':',' ','negative')

In [76]:
df_training['TEXT'] = df_training['TEXT'].apply(clean_text_data)
df_testing['TEXT'] = df_testing['TEXT'].apply(clean_text_data)

In [77]:
#Function to do Feature Scaling
def standardize_features(X_train,X_test, standardize=True):
    """
    Returns standardized features
    :param X_train: Training data to be standardized
    :param X_test : Testing data to be standardized
    :param standardize : A flag to indicate if we need data standardized
    :return: X_train_std,X_test_std: Standardized training and testing data
    """
    standardizer = StandardScaler()
    X_train_std = X_train
    X_test_std = X_test

    if(standardize):
        X_train_std = standardizer.fit_transform(X_train)
        X_test_std = standardizer.transform(X_test)

    return X_train_std,X_test_std

In [78]:
#Prepare training and testing datasets
X_train = df_training['TEXT']
y_train = df_training.drop('TEXT',axis=1)

X_test = df_testing['TEXT']
y_test = df_testing.drop('TEXT',axis=1)

In [80]:
#Function to compute F1_Score (microaveraged)
def compute_f1_score_micro(y_true, y_predicted):
    '''
    Description : Computes and returns F1 score microaveraged
    Arguments:
                 y_true:       True value
                 y_ predicted: Predicted values 
    '''
    TP,FP,TN,FN=0,0,0,0
    TP_sum,FP_sum,TN_sum,FN_sum=0,0,0,0
    for column_index in range(y_true.shape[1]):
        true_values = np.array(y_true)[:,column_index]
        predicted_values = np.array(y_predicted)[:,column_index]
        for index in range(len(true_values)):
            if(true_values[index]==predicted_values[index]==1):
                TP += 1
            elif(true_values[index]==predicted_values[index]==-1):
                TP += 1
            elif(true_values[index]==0 and predicted_values[index]!=0):
                FP += 1
            elif(true_values[index]!=0 and predicted_values[index]==0):
                FN += 1
            else:
                TN += 1   
        TP_sum = TP_sum + TP
        FP_sum = FP_sum + FP
        TN_sum = TN_sum + TN
        FN_sum = FN_sum + FN
    return ((2*TP_sum)/(FP_sum+FN_sum+(2*TP_sum)))

In [81]:
#Function to compute accuracy of polarity prediction
def compute_polarity_accuracy_score(y_true, y_predicted):
    '''
    Description : Compute the accuracy of the polarity
    Arguments : 
                   y_true :   True value
                   y_ predicted: Predicted values 
    '''
    correct = 0
    total = 0
    for column_index in range(y_true.shape[1]):
        true_values = np.array(y_true)[:,column_index]
        predicted_values = np.array(y_predicted)[:,column_index]
        for index in range(len(true_values)):
            if(true_values[index]==predicted_values[index]==1):
                correct += 1
                total += 1
            elif(true_values[index]==predicted_values[index]==-1):
                correct += 1
                total += 1
            elif(true_values[index]==1 and predicted_values[index]==-1):
                total += 1
            elif(true_values[index]==-1 and predicted_values[index]==1):
                total += 1
            else:
                pass
    return (correct/total)

In [82]:
#Make scoring function
f1_scorer = make_scorer(compute_f1_score_micro, greater_is_better=True)

### 3) Baseline Classifer (RandomForest with CountVectorizer)

In [83]:
#RandomForest Classifier
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(max_depth=3,class_weight='balanced')))])
#RandomForest specific parameters
parameters = {
'clf__estimator__n_estimators': [100,300,350]
}
#Find the optimal parameters for RandomForest
model_parameter_selection = GridSearchCV(pipeline,param_grid=parameters,cv=5,scoring = f1_scorer)
model_parameter_selection.fit(X_train, y_train)
print("Best Estimator Parameters are:",model_parameter_selection.best_params_, model_parameter_selection.best_score_)
y_predicted = model_parameter_selection.predict(X_test)
f1_micro=compute_f1_score_micro(y_test,y_predicted)
polarity_accuracy = compute_polarity_accuracy_score(y_test,y_predicted)

Best Estimator Parameters are: {'clf__estimator__n_estimators': 300} 0.728430752037


In [84]:
print('F1-Score for classifier: ',f1_micro)
print('Accuracy Score for sentiment polarity: ',polarity_accuracy)

F1-Score for classifier:  0.7286890718896061
Accuracy Score for sentiment polarity:  0.8647540983606558


In [85]:
#Function to find aspects and sentiment polarity given a review
def analyze_review(classifier, review_text):
    '''
    Description : Detects Aspects and finds the polarities given review text
    Arguments : 
                review_text : A string with review sentence/sentences
                classifier: A model used to predict the aspects and polarities
    '''
    dictionary = {'review':clean_text_data(review_text)}
    df_input=pd.DataFrame(dictionary,index=np.arange(len(dictionary.keys())))
    output = classifier.predict(df_input['review'])
    output_to_labels(output)
    

In [86]:
classifier = model_parameter_selection
review_text = "great food; bad ambiance"

In [87]:
print('F1-Score for classifier: ',f1_micro)
print('Accuracy Score for sentiment polarity: ',polarity_accuracy)

F1-Score for classifier:  0.7286890718896061
Accuracy Score for sentiment polarity:  0.8647540983606558


In [88]:
import nltk

In [89]:
nltk1=lambda x: nltk.word_tokenize(x)
nltk2=lambda x: nltk.pos_tag(x)

In [90]:
df_training['NLTK1']=df_training.TEXT.apply(nltk1)

In [91]:
df_training['NLTK2']=df_training['NLTK1'].apply(nltk2)
df_training['NLTK3']=df_training['NLTK1'].apply(nltk2)

In [92]:
df_testing['NLTK1']=df_testing.TEXT.apply(nltk1)
df_testing['NLTK2']=df_testing['NLTK1'].apply(nltk2)
df_testing['NLTK3']=df_testing['NLTK1'].apply(nltk2)

In [93]:
df_testing.shape
#df_training.shape

(90, 16)

In [94]:
for i in range(0,335):
    s = ""
    for j in range(0,len(df_training['TEXT'][i].split())):
        (word,pos)=df_training['NLTK2'][i][j]
        s+=str(pos)
        s+=str(" ")
        s+=str(word)
        s+=str(" ")
    
    df_training['NLTK3'][i]=s
   

C:\Users\atharp\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [95]:
for i in range(0,90):
    s = ""
    for j in range(0,len(df_testing['TEXT'][i].split())):
        (word,pos)=df_testing['NLTK2'][i][j]
        s+=str(pos)
        s+=str(" ")
        s+=str(word)
        s+=str(" ")
    
    df_testing['NLTK3'][i]=s
    

C:\Users\atharp\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [99]:
#Prepare training and testing datasets
X_train = df_training['NLTK3']
X_test = df_testing['NLTK3']

In [97]:
#RandomForest Classifier
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(max_depth=3,class_weight='balanced')))])
#RandomForest specific parameters
parameters = {
'clf__estimator__n_estimators': [100,300,350]
}
#Find the optimal parameters for RandomForest
model_parameter_selection = GridSearchCV(pipeline,param_grid=parameters,cv=5,scoring = f1_scorer)
model_parameter_selection.fit(X_train, y_train)
print("Best Estimator Parameters are:",model_parameter_selection.best_params_, model_parameter_selection.best_score_)
y_predicted = model_parameter_selection.predict(X_test)
f1_micro=compute_f1_score_micro(y_test,y_predicted)
polarity_accuracy = compute_polarity_accuracy_score(y_test,y_predicted)

Best Estimator Parameters are: {'clf__estimator__n_estimators': 350} 0.716905740249


In [98]:
print('F1-Score for classifier: ',f1_micro)
print('Accuracy Score for sentiment polarity: ',polarity_accuracy)

F1-Score for classifier:  0.7176969426467307
Accuracy Score for sentiment polarity:  0.8333333333333334
